## 2. 分割数据集
数据的预处理在DIN_data_processing文件已经完成，这里直接使用处理后的文件remap.pkl

In [19]:
import pickle
import pandas as pd
import numpy as np
import pickle
import random

from tensorflow.python.keras.callbacks import EarlyStopping
from tqdm import tqdm
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [20]:
def sparseFeature(feat, feat_num, embed_dim=4):
    """
    create dictionary for sparse feature
    :param feat: feature name
    :param feat_num: the total number of sparse features that do not repeat
    :param embed_dim: embedding dimension
    :return:
    """
    return {'feat': feat, 'feat_num': feat_num, 'embed_dim': embed_dim}

In [21]:
def denseFeature(feat):
    """
    create dictionary for dense feature
    :param feat: dense feature name
    :return:
    """
    return {'feat': feat}


In [22]:
def create_amazon_electronic_dataset(file, embed_dim=8, maxlen=40):
    """
    :param file: dataset path
    :param embed_dim: latent factor
    :param maxlen:
    :return: user_num, item_num, train_df, test_df
    """
    print('开始处理数据')
    # 依次读取存到pkl文件的数据
    with open(file, 'rb') as f:
        reviews_df = pickle.load(f)
        cate_list = pickle.load(f)
        user_count, item_count, cate_count, example_count = pickle.load(f)

    # 指定行为数据的列名
    reviews_df = reviews_df
    reviews_df.columns = ['user_id', 'item_id', 'time']

    # 训练集，验证集，测试集
    train_data, val_data, test_data = [], [], []

    """
    生成训练集、测试集，每个用户所有浏览的物品（共n个）前n-2个为训练集（正样本），
    并生成相应的负样本，每个用户
    共有n-3个训练集（第1个无浏览历史），第n个作为测试集。
    """
    for user_id, hist in tqdm(reviews_df.groupby('user_id')):
        # 正例商品列表
        pos_list = hist['item_id'].tolist()

        # 生成反例的方法
        def gen_neg():
            neg = pos_list[0]
            while neg in pos_list:
                neg = random.randint(0, item_count - 1)
            return neg

        # 生成和正例个数相同的反例
        neg_list = [gen_neg() for i in range(len(pos_list))]

        hist = []
        # 从第二个正例（下标为1）开始生成历史记录（第一个正例没有历史记录）
        for i in range(1, len(pos_list)):
            # 把上一次访问的商品做为浏览历史，存入hist，格式为[商品id,所属品类]
            hist.append([pos_list[i - 1], cate_list[pos_list[i - 1]]])
            # 历史记录
            hist_i = hist.copy()
            # 如果是最后一个样本，放入测试集
            # 格式为[]
            if i == len(pos_list) - 1:
                """
                    保存的正负样本的格式为：
                    [浏览历史, [正/负样本物品id, 样本所属类别], 标签]
                    其中浏览历史是多个[物品id，类别]的列表
                """
                test_data.append(
                    [hist_i, [pos_list[i], cate_list[pos_list[i]]], 1]
                )
                test_data.append(
                    [hist_i, [neg_list[i], cate_list[neg_list[i]]], 0]
                )
            # 倒数第二个做验证集
            elif i == len(pos_list) - 2:
                val_data.append(
                    [hist_i, [pos_list[i], cate_list[pos_list[i]]], 1]
                )
                val_data.append(
                    [hist_i, [neg_list[i], cate_list[neg_list[i]]], 0]
                )
            # 其他的作训练集
            else:
                train_data.append(
                    [hist_i, [pos_list[i], cate_list[pos_list[i]]], 1]
                )
                train_data.append(
                    [hist_i, [neg_list[i], cate_list[neg_list[i]]], 0]
                )

    # 特征列，包含Dense和Sparse
    feature_columns = [
        [],
        [sparseFeature('item_id', item_count, embed_dim), ]
    ]

    # 行为数据
    behavior_list = ['item_id']

    # 打乱数据
    random.shuffle(train_data)
    random.shuffle(val_data)
    random.shuffle(test_data)

    # 把训练集，验证集，测试集转为DataFrame
    train = pd.DataFrame(train_data, columns=['hist', 'target_item', 'label'])
    val = pd.DataFrame(val_data, columns=['hist', 'target_item', 'label'])
    test = pd.DataFrame(test_data, columns=['hist', 'target_item', 'label'])

    train_X = [
        np.array([0.] * len(train)),  # 长度为训练集长度的全0.列表
        np.array([0] * len(train)),  # 长度为训练集长度的全0列表
        pad_sequences(train['hist'], maxlen=maxlen),  # 把历史浏览统一填充为最大长度
        np.array(train['target_item'].tolist())  # 目标商品
    ]
    train_y = train['label'].values  # 训练集标签
    val_X = [
        np.array([0] * len(val)), np.array([0] * len(val)),
        pad_sequences(val['hist'], maxlen=maxlen),
        np.array(val['target_item'].tolist())
    ]
    val_y = val['label'].values
    test_X = [
        np.array([0] * len(test)), np.array([0] * len(test)),
        pad_sequences(test['hist'], maxlen=maxlen),
        np.array(test['target_item'].tolist())
    ]
    test_y = test['label'].values
    print('数据处理完成')

    return feature_columns, behavior_list, (train_X, train_y), (val_X, val_y), (test_X, test_y)

## 3. 模型
### 3.1 Activation Unit

In [23]:
from tensorflow.keras.layers import Layer, Softmax, PReLU, Dense
import tensorflow as tf

# 限制GPU使用量
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]
)

In [24]:
# 注意力层，是整个注意力网络的操作，包含所有注意力单元
class Activation_Layer(Layer):
    def __init__(self, att_hidden_units, activation='prelu'):
        """

        :param att_hidden_units:
        :param activation:
        """
        super().__init__()
        self.att_denses = [
            Dense(unit, activation=activation)
            for unit in att_hidden_units
        ]
        self.linear = Dense(1)

    def call(self, inputs, **kwargs):
        # q, k, v都是注意力机制里的东西
        # query: 候选项(Ad A)  (None, d * l), d 是embedding后的维度, l是 历史行为列的列数
        # key: hist items  (None, seq_len, d * l)
        # value: hist items  (None, seq_len, d * l)
        # mask: (None, seq_len)
        q, k, v, mask = inputs
        # tile用来复制拓展张量，这一步把query矩阵复制seq_len次
        q = tf.tile(q, multiples=[1, k.shape[1]])  # (None, seq_len * d * 2)
        # 再把形状转化为seq_len行，dim * l 列的矩阵
        q = tf.reshape(q, shape=[-1, k.shape[1], k.shape[2]])  # (None, seq_len, d * l)

        # 这个地方就是激活单元中所谓的外积，其实就是去了差和乘积（不是外积）
        info = tf.concat([q - k, q * k], axis=-1)
        # 再和原始输入做拼接
        info = tf.concat([q, k, info], axis=-1)

        # 多个全连接层的计算
        for dense in self.att_denses:
            info = dense(info)

        # 输出层计算
        out = self.linear(info)  # (None, seq_len, 1)
        out = tf.squeeze(out, axis=-1)  # (None, seq_len)

        # 为padding的mask补一个很小的负数，这样后面计算 softmax 时, e^{x} 结果就约等于 0
        # ones_like 创建一个将所有元素都设置为1的张量（给定一个张量，使用该张量的形状生成全1张量）
        # tf.where 根据condition返回x和y中的元素
        padding = tf.ones_like(out) * (-2 ** 32 + 1)  # (None, seq_len)
        out = tf.where(tf.equal(mask, 0), padding, out)  # (None, seq_len)

        out = tf.nn.softmax(out)  # (None, seq_len)
        # 转成二维的
        out = tf.expand_dims(out, axis=1)  # (None,1, seq_len)

        out = tf.matmul(out, v)  #  (None, seq_len, d * 2) *  # (None,1, seq_len) = # (None, 1, d * 2)
        out = tf.squeeze(out, axis=1)  # # (None, d * 2)

        return out


class Dice(Layer):
    def __init__(self):
        super(Dice, self).__init__()
        self.bn = BatchNormalization(center=False, scale=False)
        self.alpha = self.add_weight(shape=(), dtype=tf.float32, name='alpha')

    def call(self, x):
        x_normed = self.bn(x)
        x_p = tf.sigmoid(x_normed)

        return self.alpha * (1.0 - x_p) * x + x_p * x

### 3.2 DIN

In [25]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Embedding, PReLU, Dropout, Input, BatchNormalization
from tensorflow.keras.regularizers import l2


class DIN(Model):
    def __init__(self, feature_columns, behavior_feature_list, att_hidden_units=(80, 40), att_activation='prelu',
                 ffn_hidden_units=(80, 40), ffn_activation='prelu', maxlen=40, dnn_dropout=0., embed_reg=1e-4):
        super(DIN, self).__init__()
        self.maxlen = maxlen
        self.dense_feature_columns, self.sparse_feature_columns = feature_columns
        # 行为数据列表
        self.behavior_feature_list = behavior_feature_list
        # sparse特征中不是行为特征的列数
        self.other_sparse_len = len(self.sparse_feature_columns) - len(behavior_feature_list)
        # Dense特征列数
        self.dense_len = len(self.dense_feature_columns)
        # 行为特征的列数
        self.behavior_num = len(behavior_feature_list)

        # 因为历史行为和其他特征处理方式不一样，所以需要把两种特征分开做Embedding
        # 行为特征的Embedding
        self.behavior_embed_layers = [
            Embedding(
                input_dim=feat['feat_num'],
                input_length=1,
                output_dim=feat['embed_dim'],
                embeddings_initializer='random_uniform',
                embeddings_regularizer=l2(embed_reg)
            )
            for feat in self.sparse_feature_columns
            if feat in self.behavior_feature_list
        ]
        # 其他特征的Embedding
        self.sparse_embed_layers = [
            Embedding(
                input_dim=feat['feat_num'],
                input_length=1,
                output_dim=feat['embed_dim'],
                embeddings_initializer='random_uniform',
                embeddings_regularizer=l2(embed_reg)
            )
            for feat in self.sparse_feature_columns
            if feat not in self.behavior_feature_list
        ]

        # 注意力网络
        self.attention_layer = Activation_Layer(att_hidden_units, att_activation)

        #?
        self.bn = BatchNormalization(trainable=True)

        # 一个多层的PRelu/Dice激活的MLP网络
        self.ffn = [
            Dense(unit, activation=PReLU() if ffn_activation == 'prelu' else Dice())
            for unit in ffn_hidden_units
        ]
        # Dropout层
        self.dropout = Dropout(dnn_dropout)
        # 输出层
        self.dense_final = Dense(1)

    def call(self, inputs):
        """

        :param inputs: dense_feature, spare_feature, seq_inputs, item_inputs
        :return:
        """
        '''
            这里的inputs对应输入的一条样本：
            dense_feature = [0, 0, ..., 0]
            sparse_feature = [0., 0., ..., 0.]
            seq_inputs = 浏览历史   (None, maxlen, behavior_num)
            item_inputs = 广告物品  (None, behavior_num)
        '''
        # print(inputs)
        dense_feature, spare_feature, seq_inputs, item_inputs = inputs

        # 把历史行为数据中item_id不为0的转为float类型
        mask = tf.cast(tf.not_equal(seq_inputs[:, :, 0], 0), dtype=tf.float32)  # (None, maxlen)

        # 把Dense复制一份
        other_info = dense_feature

        # 做Embedding（循环对sparse每一项做embed）
        for i in range(self.other_sparse_len):
            other_info = tf.concat(
                [other_info, self.sparse_embed_layers[i](spare_feature[:, i])],
                axis=-1
            )

        # 给历史浏览记录做embed（依此给每一个商品做embedding）
        seq_embed = tf.concat(
            [
                self.sparse_embed_layers[i](seq_inputs[:, :, i])
                for i in range(self.behavior_num)
            ],
            axis=-1
        )  #

        # 给候选广告做embed
        item_embed = tf.concat(
            [
                self.sparse_embed_layers[i](item_inputs[:, i])
                for i in range(self.behavior_num)
            ],
            axis=-1
        )

        '''
            注意力网络
            item_embed： query 候选物品
            seq_embed： key 浏览历史
            seq_embed: value 浏览历史
            mask: mask
        '''
        # 注意力层的计算
        user_info = self.attention_layer([item_embed, seq_embed, seq_embed, mask])  # (None, d * l)

        # 连接层，把所有特征连结
        if self.dense_len > 0 or self.other_sparse_len > 0:
            info_all = tf.concat(
                [user_info, item_embed, other_info],
                axis=-1
            )
        else:
            info_all = tf.concat(
                [user_info, item_embed],
                axis=-1
            )

        # 。
        info_all = self.bn(info_all)

        # ffn计算
        for dense in self.ffn:
            info_all = dense(info_all)

        # dropout
        info_all = self.dropout(info_all)
        # 最终输出
        outputs = tf.nn.sigmoid(self.dense_final(info_all))
        return outputs

    def summary(self, line_length=None, positions=None, print_fn=None):
        dense_inputs = Input(
            shape=(self.dense_len,),
            dtype=tf.float32
        )
        sparse_inputs = Input(
            shape=(self.other_sparse_len,),
            dtype=tf.int32
        )
        seq_inputs = Input(
            shape=(self.maxlen, self.behavior_num),
            dtype=tf.int32
        )
        item_inputs = Input(
            shape=(self.behavior_num,),
            dtype=tf.int32
        )
        tf.keras.Model(
            inputs=[dense_inputs, sparse_inputs, seq_inputs, item_inputs],
            outputs=self.call([dense_inputs, sparse_inputs, seq_inputs, item_inputs])
        ).summary()

In [26]:
def test_model():
    dense_features = [
        {'feat': 'a'}, {'feat': 'b'}
    ]

    sparse_features = [
        {'feat': 'item_id', 'feat_num': 100, 'embed_dim': 8},
        {'feat': 'cate_id', 'feat_num': 100, 'embed_dim': 8},
        {'feat': 'sss_id', 'feat_num': 100, 'embed_dim': 8},
        {'feat': 'adv_id', 'feat_num': 100, 'embed_dim': 8}
    ]

    behavior_list = ['item_id', 'cate_id', 'sss_id']

    features = [dense_features, sparse_features]

    model = DIN(features, behavior_list)

    model.summary()



In [27]:
test_model()

ValueError: Unknown activation function: prelu

## 4. 训练模型

In [28]:
from time import time
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC

### 4.1 设置参数

In [29]:
file = 'E:/master/paper/data/DIN_data/remap.pkl'
maxlen = 20

embed_dim = 8
att_hidden_units = [80, 40]
ffn_hidden_units = [256.128, 64]
dnn_dropout = 0.5
att_activation = 'sigmoid'
ffn_activation = 'prelu'

learning_rate = 0.001
batch_size = 4096
epochs = 20

### 4.2 创建数据集

In [30]:
# 特征列，行为数据列表，训练集，验证集，测试集
feature_columns, behavior_list, train, val, test = create_amazon_electronic_dataset(file, embed_dim, maxlen)
train_X, train_y = train
val_X, val_y = val
test_X, test_y = test

开始处理数据


100%|██████████| 192403/192403 [00:36<00:00, 5291.63it/s]


数据处理完成


### 4.3 创建模型

In [31]:
# 创建模型
model = DIN(
    feature_columns=feature_columns,
    behavior_feature_list=behavior_list,
    att_hidden_units=att_hidden_units,
    att_activation=att_activation,
    ffn_hidden_units=ffn_hidden_units,
    ffn_activation=ffn_activation,
    dnn_dropout=dnn_dropout
)

model.compile(
    loss=binary_crossentropy,
    optimizer=Adam(learning_rate),
    metrics=[AUC()]
)

### 4.4 训练模型

In [32]:
import os
from _datetime import datetime

log_dir = os.path.join("../logs/din-" + datetime.now().strftime("%Y%m%d-%H%M%S"))
callback = tf.keras.callbacks.TensorBoard(log_dir, histogram_freq=1)

model.fit(
    train_X,
    train_y,
    epochs=epochs,
    callbacks=[callback],
    # callbacks=EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True),
    validation_data=(val_X, val_y),
    batch_size=batch_size
)

Epoch 1/20
543/543 [==============================] - 12s 19ms/step - loss: 0.6297 - auc_1: 0.7095 - val_loss: 0.6038 - val_auc_1: 0.7891

In [ ]:
print('test AUC: %f' % model.evaluate(test_X, test_y, batch_size=batch_size)[1])